In [1]:
import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)   
# pd.set_option('display.max_colwidth', None) 


import snowflake.connector
import os
import pandas as pd

# ENTER YOUR SNOWFLAKE CREDENTIALS
# If you don't have them here, or as environment variables, you'll be prompted on run.
SNOWFLAKE_USER = "" 
SNOWFLAKE_PASSWORD = ""

if SNOWFLAKE_USER == "":
    SNOWFLAKE_USER = os.environ.get("SNOWFLAKE_USER")
    if not SNOWFLAKE_USER or SNOWFLAKE_USER == "":
        SNOWFLAKE_USER = input("Enter your Snowflake username: ")
if SNOWFLAKE_PASSWORD == "":
    SNOWFLAKE_PASSWORD = os.environ.get("SNOWFLAKE_PASSWORD")
    if not SNOWFLAKE_PASSWORD or SNOWFLAKE_PASSWORD == "":
        SNOWFLAKE_PASSWORD = input("Enter your Snowflake password: ")

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account='fougxvx-fb30115',
    warehouse='BOTANALYTICS_WH',
)

# Create a cursor object
cur = conn.cursor()



In [ ]:
from datetime import timedelta
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)   
pd.set_option('display.max_colwidth', None) 


# df = pd.read_csv('testerResultsNBF.csv')
 # where site_id in ('walmart-tx-newbraunfels-site1') 
cur.execute(f"""
            select BOT_ID, site_id,DATEADD(HOUR,-5,DATETIME) as datetime, storeason, hierarchy_component, event, subevent, actionecode, swversion, location_x, location_y,location_level, filename, velocity_linear, actions_0,transfertype
            from SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
           
            where site_id in ('walmart-tx-newbraunfels-site1') 
           
                AND event = 'collisionDetected'
                AND location_level != 0
                AND DATEADD(HOUR, -5, DATETIME) >= '2025-07-28 00:00:00' 
                AND DATEADD(HOUR, -5, DATETIME) <= '2025-07-29 00:00:00'
            order by datetime
        """)

rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]

df = pd.DataFrame(rows, columns=cols)


# df = df[df["SWVERSION"] == '3.161.0-cas_enabled']

print(type(df['LOCATION_Y']))
df['LOCATION_Y'] = pd.to_numeric(df['LOCATION_Y'], errors='coerce')
df['LOCATION_X'] = pd.to_numeric(df['LOCATION_X'], errors='coerce')

# df = df[(df['LOCATION_Y'] >= 0) & (df['LOCATION_Y'] <= 8)]
df['DATETIME'] = pd.to_datetime(df['DATETIME'], errors='coerce')
df['DAY'] = df['DATETIME'].dt.strftime('%B %#d') 


df = df.sort_values(by='DATETIME')

df['GROUP_ID'] = np.nan  

def is_close(instance1, instance2):
    if(instance1['SUBEVENT'] == 'highMagnitudeCollision' and instance2['SUBEVENT'] == 'highMagnitudeCollision'):
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=300) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 5.0 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 5.0 and
        instance1['BOT_ID'] != instance2['BOT_ID']
    )
    elif(instance1['SUBEVENT'] == 'highMagnitudeCollision' or instance2['SUBEVENT'] == 'highMagnitudeCollision'):
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=120) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 2.5 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 2.5 and
        instance1['BOT_ID'] != instance2['BOT_ID']
    )
    else:
        return (
        abs(instance1['DATETIME'] - instance2['DATETIME']) <= timedelta(seconds=60) and
        abs(instance1['LOCATION_X'] - instance2['LOCATION_X']) <= 2.5 and
        abs(instance1['LOCATION_Y'] - instance2['LOCATION_Y']) <= 2.5 and
        instance1['BOT_ID'] != instance2['BOT_ID']
        )

    

group_id = 1

for day in df['DAY'].unique():
    for level in df['LOCATION_LEVEL'].dropna().unique():
        dfFiltered = df[(df['DAY'] == day) & (df['LOCATION_LEVEL'] == level)].copy()
        dfFiltered.reset_index(inplace=True)

        seen = set()

        for idx, row in dfFiltered.iterrows():
            if idx in seen:
                continue

            group = [idx]
            queue = [idx]
            seen.add(idx)

            while queue:
                current_idx = queue.pop()
                current_row = dfFiltered.loc[current_idx]

                for other_idx, other_row in dfFiltered.iterrows():
                    if other_idx in seen:
                        continue
                    if is_close(current_row, other_row):
                        group.append(other_idx)
                        queue.append(other_idx)
                        seen.add(other_idx)
 
            if len(set(dfFiltered.loc[group, 'BOT_ID'])) > 1:
                df.loc[dfFiltered.loc[group, 'index'], 'GROUP_ID'] = group_id
                group_id += 1
            # else:
            #     if dfFiltered.loc[group, 'SUBEVENT'] == 'highMagnitudeCollision':
            #         df.loc[dfFiltered.loc[group, 'index'], 'GROUP_ID'] = 'none'


result_df = df[df['GROUP_ID'].notna()].copy()


for i in range(1, group_id + 1):
    dfGroup = result_df[result_df['GROUP_ID'] == i]
    if 'highMagnitudeCollision' not in dfGroup['SUBEVENT'].values:
            result_df = result_df[result_df['GROUP_ID'] != i]

result_df = result_df.sort_values(
    by=['GROUP_ID', 'SUBEVENT'],
    key=lambda col: (
        col if col.name != 'SUBEVENT' else col != 'highMagnitudeCollision'  
    )
)

result_df = result_df.drop_duplicates(subset=['BOT_ID', 'GROUP_ID'])
result_df[['BOT_ID', 'GROUP_ID', 'DATETIME', 'LOCATION_X', 'LOCATION_Y', 'LOCATION_LEVEL', 'DAY']]
result_df.reset_index(drop=True)



result_df


<class 'pandas.core.series.Series'>


,BOT_ID,SITE_ID,DATETIME,STOREASON,HIERARCHY_COMPONENT,EVENT,SUBEVENT,ACTIONECODE,SWVERSION,LOCATION_X,LOCATION_Y,LOCATION_LEVEL,FILENAME,VELOCITY_LINEAR,ACTIONS_0,TRANSFERTYPE,DAY,GROUP_ID
0,8375,walmart-tx-newbraunfels-site1,2025-07-31 04:32:17.746,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,0.408194,4.218872,3,bot_8375_20250731_093313_UTC.ion,4.0106215477,1027700,None,July 31,4.0
1,32725,walmart-tx-newbraunfels-site1,2025-07-31 04:32:18.225,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,-0.921605,5.418649,3,bot_32725_20250731_093246_UTC.ion,1.8291829824,1061490,None,July 31,4.0
2,22260,walmart-tx-newbraunfels-site1,2025-07-31 05:35:34.346,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,32.990528,5.037320,4,bot_22260_20250731_103703_UTC.ion,-0.32487607,1025470,None,July 31,8.0
3,22461,walmart-tx-newbraunfels-site1,2025-07-31 05:35:34.458,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,32.516327,6.280523,4,bot_22461_20250731_103628_UTC.ion,1.2969522476,1070723,None,July 31,8.0
4,32149,walmart-tx-newbraunfels-site1,2025-07-31 13:35:57.299,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,34.101002,0.080964,6,bot_32149_20250731_183641_UTC.ion,0.9868531823,1083687,None,July 31,1.0
5,31659,walmart-tx-newbraunfels-site1,2025-07-31 13:36:08.668,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,34.101002,0.230908,6,bot_31659_20250731_183651_UTC.ion,0.9110225439,1078023,None,July 31,1.0
6,8202,walmart-tx-newbraunfels-site1,2025-07-31 14:13:49.555,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,-124.194000,7.258420,2,bot_8202_20250731_191447_UTC.ion,-0.0031959915,1077134,None,July 31,2.0
7,9092,walmart-tx-newbraunfels-site1,2025-07-31 14:14:10.917,None,POSCNTRL,collisionDetected,lowMagnitudeCollision,None,3.171.0-nbf,-126.269997,7.273856,2,bot_9092_20250731_191456_UTC.ion,-0.0159458127,1080524,None,July 31,2.0
8,9092,walmart-tx-newbraunfels-site1,2025-07-31 15:19:35.833,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-13.781755,2.342130,2,bot_9092_20250731_201959_UTC.ion,1.9855546951,1082101,None,July 31,3.0
9,24124,walmart-tx-newbraunfels-site1,2025-07-31 15:19:36.137,None,POSCNTRL,collisionDetected,highMagnitudeCollision,None,3.171.0-nbf,-11.712789,1.174626,2,bot_24124_20250731_202043_UTC.ion,0,1080011,None,July 31,3.0


In [ ]:
## RUNNING THIS CELL WILL TELL YOU WHAT GROUPING IDS HAVE AT LEAST ONE BOT WITH A STUCK ALARM BUT IT ISN'T NECESSARY

groupKeeper = set()
for _,rows1 in result_df.iterrows():
    if(rows1['GROUP_ID'] not in groupKeeper):
        bot = rows1['BOT_ID']
        start = rows1['DATETIME'] - pd.Timedelta(minutes=5)
        end = rows1['DATETIME'] + pd.Timedelta(minutes=5)
        cur.execute(f"""
            select BOT_ID, site_id,DATEADD(HOUR,-5,DATETIME) as datetime, storeason, hierarchy_component, event, subevent, actionecode, swversion, location_x, location_y,location_level, filename, velocity_linear, actions_0,transfertype

            from SYMBOTIC_KAFKA_DB.LOG_TABLES.DIAGNOSTICMESSAGE
            where site_id in ('walmart-tx-newbraunfels-site1') 
                And bot_id = '{bot}'

                AND event = 'alarm'
                AND location_level != 0
                AND DATEADD(HOUR, -5, DATETIME) >= '{start}'
                AND DATEADD(HOUR, -5, DATETIME) <= '{end}'
            order by datetime
        """)

        rows = cur.fetchall()
        cols = [desc[0] for desc in cur.description]

        newDf = pd.DataFrame(rows, columns=cols)
        
        val = False
        for _,newRows in newDf.iterrows():
            if(newRows['SUBEVENT'] == 'stuck' or newRows['SUBEVENT'] == 'estimation_error'):
                val = True
        
        if(val):
            groupKeeper.add(rows1['GROUP_ID'])
    

## RETURNS THE GROUP IDS WITH STUCK ALARMS IN AT LEAST ONE BOT
groupKeeper




